In [1]:
import pandas as pd
import numpy as np
import konlpy
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import re
import random
import numpy as np

okt = Okt()
komoran=Komoran()
kkma=Kkma()
hannanum=Hannanum()

In [2]:
df = pd.read_excel("/Users/young/git/project/export/비전공자 개발자_2023-01-31.xlsx")
df.rename(columns={'Unnamed: 0':'Rank'},inplace=True)


In [3]:
df.tail(5)

,Rank,title,description,author,timestamp
1045,1045,"문과 개발자, 정말 비전공자도 IT 취업이 가능할까?","문과 개발자, 정말 비전공자도 IT 취업이 가능할까? 문과라서 취업할 길이 보이...",그대 미소에 봄이 돼줄게요,2021.09.18.
1046,1046,"자바 개발자 되는 법, 비전공자에게 적합한 과정은?",자바 개발자 되는 법 비전공자에게 맞는 과정 자바 개발자가 되기 위해 비전공자에...,IT교육및취업전문가 융쌤,2022.06.03.
1047,1047,"소프트웨어개발자되는법, 컴공출신 아닌 비전공자 자바웹포폴...",오늘은 컴공전공생이 아닌 비전공자가 #소프트웨어개발자되는법 #웹개발프로젝트 제작...,월요일에 만든 블로그,2022.01.20.
1048,1048,종로자바학원에서 비전공자도 개발자 취업 가능해요!!,종로자바학원에서 비전공자도 개발자 취업 가능해요!! 비전공자분들의 가장 큰 고민...,따봉아빠의 IT 일상,2017.09.08.
1049,1049,(비전공자 공부순서) 개발자가 하고 싶어요.....,그럼에도 불구하고 디지털노마드를 꿈꾸시거나 개발자의 꿈을 갖고 계신 분 혹은.....,초보개발자 _ 왕곰,2022.09.13.


In [5]:
df['structure'] = ''

In [6]:
def pos_seperator(pos):
    #품사
    pos = re.search(('(?<=\/)[a-zA-Z]*'),tup)[0]
    #형태소
    morphs = re.search(('.+(?=\/)'),tup)[0]

def add_scores():
    return 

def word_parser(df):
    morphs_list = []
    phrase_list = []
    structure_list = []
    # count_morphs_list = []
    # count_phrase_list = []
    # df 반복문 돌며 어절, 형태소 분해, 문장구조 분해 
    for idx,rows in df.iterrows():
        split_phrases = okt.phrases(rows[1])
        split_morphs = okt.pos(rows[1], join=True)

        for i in split_phrases:
            phrase_list.append(i + '/' + 'Noun')
        for i in split_morphs:
            if 'Punctuation' in i:
                continue
            morphs_list.append(i)
    #명사 중복 제거 후 구조화
        # tmp_structure = []
        tmp_structure = ''
        for jdx,tup in enumerate(okt.pos(rows[1])):
            if jdx > 0 and tmp == 'Noun' and tup[1] == 'Noun':
                continue
            tmp = okt.pos(rows[1])[jdx][1]
            # structure_list.append(tup[1])
            # tmp_structure += '_' + tup[1] if jdx != 0 or jdx != len(okt.pos(rows[1])) - 1 else tup[1]
            # if jdx != 0:
            #     tmp_structure += '_' + tup[1]
            # else:
            #     tmp_structure += tup[1]
            tmp_structure += '_' + tup[1]
            # structure_list.append([i for i in tup[1]])
        structure_list.append(tmp_structure)

    #unique값 찾기
    morphs_set = list(set(morphs_list))
    phrase_set = list(set(phrase_list))
    structure_set = list(set(structure_list))
    #빈도 카운트
    print(len(phrase_set))
    for i in range(len(morphs_set)):
        #두 리스트간 중복 삭제
        if morphs_set[i] in phrase_set:
            phrase_set.remove(morphs_set[i])
        morphs_set[i] = morphs_set[i] + '/' + str(morphs_list.count(morphs_set[i]))

    for i in range(len(phrase_set)):
        phrase_set[i] = phrase_set[i] + '_joint/' + str(phrase_list.count(phrase_set[i]))

    for i in range(len(structure_set)):
        structure_set[i] = structure_set[i] + '/' + str(structure_list.count(structure_set[i]))

    return phrase_set,morphs_set,structure_set,structure_list

phrase_set,morphs_set,structure_set,structure_list = word_parser(df)



3251


In [7]:
df_orgin_structure = []
for i in structure_list:
    
    split = re.split(('_'),i)
    del split[0]

    df_orgin_structure.append(split)

df['structure'] = df_orgin_structure



In [8]:
#단어 토큰 df화
frame = {
    'word' : [],
    'pos' : [],
    'cnt' : []
}
word_st = phrase_set + morphs_set
for i in word_st:
    try:
        split = re.split(('/'),i)
        # print(split[1],split[2],split[3])
        frame['word'].append(split[0])
        frame['pos'].append(split[1])
        frame['cnt'].append(int(split[2]))
    except:
        # print(i)
        continue

frame
df2 = pd.DataFrame(frame)
df2 = df2.sort_values(by="cnt",ascending=False)
df2

,word,pos,cnt
3144,비,Noun,1063
2675,전공자,Noun,1062
3923,개발자,Noun,1029
2262,비전공자,Noun_joint,659
3312,취업,Noun,599
...,...,...,...
1430,디지털전환,Noun_joint,1
1432,스마트폰앱개발자,Noun_joint,1
1433,제46회 SQL 개발자 SQLD 비전공자 시험공부 방법,Noun_joint,1
1434,비전공자 개발자 취업 공부 솔직한 방향,Noun_joint,1


In [9]:
# 문장구조 df화
structure_frame = {
    'structure' : [],
    'cnt' : []
}
for i in structure_set:
    split_cnt = re.split(('/'),i)
    cnt = int(split_cnt[-1])
    
    split = re.split(('_'),i)
    del split[0]
    split[-1] = re.split(('/'),split[-1])[0]
    # print(cnt)
    structure_frame['structure'].append(split)
    structure_frame['cnt'].append(cnt)
df3 = pd.DataFrame(structure_frame)
df3 = df3.sort_values(by="cnt",ascending=False)
df3

,structure,cnt
524,[Noun],40
588,"[Noun, Punctuation, Noun]",19
251,"[Noun, Josa, Noun]",18
54,"[Noun, Punctuation]",11
305,"[Noun, Punctuation, Noun, Punctuation]",9
...,...,...
266,"[Noun, Josa, Noun, Josa, Noun, Verb, Noun, Suf...",1
267,"[Noun, Alpha, Noun, Punctuation, Noun, Verb, N...",1
269,"[Noun, Josa, Modifier, Noun, Punctuation, Noun...",1
270,"[Noun, Number, Adjective, Noun, Josa, Noun, Jo...",1


In [10]:
df3_pop = df3[df3['cnt'] > 1]

for i in range(1):
    sample_structure = random.choices(list(df3_pop['structure']),weights=[i for i in df3_pop['cnt']])[0]



In [11]:
sample_structure_dict = dict.fromkeys(sample_structure)

In [12]:
df2

,word,pos,cnt
3144,비,Noun,1063
2675,전공자,Noun,1062
3923,개발자,Noun,1029
2262,비전공자,Noun_joint,659
3312,취업,Noun,599
...,...,...,...
1430,디지털전환,Noun_joint,1
1432,스마트폰앱개발자,Noun_joint,1
1433,제46회 SQL 개발자 SQLD 비전공자 시험공부 방법,Noun_joint,1
1434,비전공자 개발자 취업 공부 솔직한 방향,Noun_joint,1


In [13]:
#문장구조와 단어 조합
def random_choice(sample_list, weight):
    sample = random.choices(sample_list,weights=weight)
    return sample[0]

for pos in sample_structure:
    if pos == 'Noun':
        coin_flip = random_choice(["Noun","Noun_joint"],[0.2,0.8])
        df2_n = df2[df2['pos'] == coin_flip]
    else:
        df2_n = df2[df2['pos'] == pos]
    df2_n.reset_index(drop=False)
    df2_n.sort_values(by='cnt',ascending=False)

    match_senetence = []
    for i,row in df.iterrows():
        if sample_structure == row[5]:
            match_senetence.append(row[1])

    words = []
    
    for i in range(100):
        if pos == 'Punctuation':
            sample = random_choice(["?","!","??","!!"],[1,1,1,1])

        else:
            sample = random_choice(list(df2_n['word']),[i for i in df2_n['cnt']])

        words.append(sample)
    sample_structure_dict[pos] = words

print(sample_structure)
print(match_senetence)
print(sample_structure_dict)

['Punctuation', 'Noun', 'Punctuation', 'Noun', 'Verb', 'Noun', 'Number']
['[스토리] 비전공자 개발자된 이야기 1', '[스토리] 비전공자 개발자된 이야기 2', '[스토리] 비전공자 개발자된 이야기 3']
{'Punctuation': ['!', '?', '??', '?', '?', '!!', '!!', '??', '!', '!', '!!', '?', '!!', '??', '!', '!', '!!', '?', '!', '!', '??', '!', '??', '!!', '!!', '??', '!', '??', '!', '??', '??', '?', '!!', '??', '??', '??', '?', '??', '!', '!', '!!', '!', '??', '!!', '?', '?', '??', '?', '??', '?', '?', '!', '?', '!!', '!', '??', '!!', '!!', '?', '!', '??', '!!', '??', '!!', '??', '??', '!', '!!', '!', '??', '!', '!', '!!', '??', '!!', '??', '??', '!', '??', '!!', '!!', '!', '!', '??', '??', '??', '?', '??', '!', '!', '??', '!!', '??', '??', '??', '!!', '!', '??', '?', '?'], 'Noun': ['취업', '자바', '개발자', '고민', '정석', '백엔드', '리뷰', '출근', '전공자', '공부', '비', '개발자', '프로그래밍', '생각', '전공자', '개발자', '인프런', '학원', '개발자', '취업', '로드맵', '저', '법', '비', '비', '전공자', '취업', '웹개발자', '취업', '방법', '전공자', '재학생', '비', '개발자', '지원', '비', '비', '지원', '말', '전공자', '개발자', '비', '말', 